# Week 5 Lab Assignment

**What is the ask?**  Implement a three-axis accelerometer using Serial Peripheral Interconnect communications and an interrupt-driven driver.

**Why is this important?** This lab brings together topics you have covered over the past month of the course to build, debug, and verify a complete system.  While your activities in this lab are still guided, you are given less support than in prior labs.  This is intentional.  You have been developing critical skills for embedded system code development, hardware prototyping, software debugging, and hardware troubleshooting.  It is time to put them to use building a practical sensor that has been the starting point for multiple commercial products.    

NOTE: I will build your code.  Make sure your code does not generate any warnings when it builds!  The penalty for code that generates warnings when it compiles is severe: 10% off!  

This is a partner-based lab but I expect you to develop, maintain, and submit individual repos.  You can submit the same code, or sit side by side and develop "highly similar" code, but where possible I ask you to provide your own (individual) code comments (more on commenting in a second).  

Where I ask you for *INDIVIDUAL RESPONSE*, however, you should answer on your own based on your own individual work/thinking.  I expect you and your partner to both implement the hardware design.  Where things go wrong with hardware I would like you to first try and troubleshoot it yourself, then work collaboratively with your partner to sort it out. 

I haven't yet gone around the room and asked you to explain your code or your hardware to me.  Be prepared for this over the next few weeks.  Don't rely on your partner's work if you do not understand it!

I *encourage* the use of AI to generate code.  Even if the code it generates works, so far what I have seen of AI-generated code will get a C from me because it is poorly commented, uses magic numbers, does not reference source documenation, and does not provide any/adequate comments about design decisions.  Use AI (or don't), but *cite* your use of it, bring it to an 'A' standard, and make sure you understand it.

Commenting: comments are where you convince me you understand what is going on... and (very) well commented code may even save you from having to explain it to me real-time!  We've gone over this quite a bit the last few weeks, so I shouldn't have to remind you... but remind you I will:

* Set expectations for your code reviewer (e.g. me!) before functions.  Describe the objective, any algorithms, input arguments, return values...
* As you implement your function, indicate where you are meeting expectations (before the function "in this function I will (1), (2), (3)"... inside of the function "here I implement (1)... here I implement (2)..."
* No magic numbers!  Use #define or, if you must use magic, explain it in a comment
* Any time you make a design decision, write it down.  If you go left, explain why you go left and why you did not go right!  
* When messing with registers, and anywhere else that it makes sense: provide a reference to the source document, document section, etc. that explains the register / the behavior resulting from changes you are making

When in doubt, ask me for clarification.  That said, this should be a very fun lab... let's get started.

## Part 1 - Serial Communications and the Serial Peripheral Interface (SPI)

The following video is necessary background on serial communications fundamentals.  I am borrowing it, out of sequence, from ENGG-462 where it is followed by a deep dive into the "universal synchronous asynchronous receiver transmitter" peripheral.  We do not do a deep dive into that in ENGS-62 (though you have been through some source code related to initializing it, sending data, and receiving data using interrupts by now).  So you can ignore the "getting ready for USART" bits in the video, but pay attention to the rest!

[Serial Communications Overview](https://dartmouth.hosted.panopto.com/Panopto/Pages/Viewer.aspx?id=10f0569a-e76c-4644-a169-b3e301082cec)

With that information onboard, let's jump into the Serial Peripheral Interconnect (SPI) peripheral with this overview video.  I also suggest starting to scan the [LIS3DH 3-Axis Accelerometer](datasheets/lis3dh.pdf), as this is the part in your lab kit you will be communicating with using SPI in this lab:

[SPI Overview](https://dartmouth.hosted.panopto.com/Panopto/Pages/Viewer.aspx?id=31c49022-a6ee-4d06-ae20-b3e30115edaa)

Now roll up your sleeves and get ready to learn about SPI interface configuration:

[SPI Configuration](https://dartmouth.hosted.panopto.com/Panopto/Pages/Viewer.aspx?id=5bd82833-4864-4d4b-a8b8-b3e30115f612)

At this point you should be ready to connect a SPI peripheral to your microcontroller... but take a minute to review the following video on some SPI debugging tips and tricks:

[SPI Debugging (without a Logic Analyzer)](https://dartmouth.hosted.panopto.com/Panopto/Pages/Viewer.aspx?id=de2fa1eb-dbb9-4a56-8233-b3e30115f659)

If you haven't looked at the Serial Peripheral Interconnect (SPI) Self Assessment in Canvas yet, this is a good time to work through it!

## Checkpoint 1 - Connecting an LIS3DH Accelerometer to the STM32F042K6 (Polling)

Let's connect a 3-axis accelerometer to the microcontroller.  You have an [LIS3DH](datasheets/lis3dh.pdf) 3-axis accelerometer in your lab kit.  It is on an [Adafruit LIS3DH Dev Board](datasheets/LIS3DH%20Adafruit%20Schematic.pdf).  It measures acceleration (e.g. Earth's gravity if sitting still on a desk, or additional accelerations on top of Earth's gravity if you move it around).  Read its description on page 1 of the datasheet.  Ready?  Let's connect it.

I am intentionally taking away a lot of the support you've had up to this point.  You've been guided through all of the bits and pieces you need.  Your task, now, is to put it together.  I'll drop some breadcrumbs along the trail, but it is up to you and your partner to find your way.  Here are links to documents you'll likely need:

* [LIS3DH Datasheet](datasheets/lis3dh.pdf)
* [Adafruit LIS3DH Dev Board Schematics](datasheets/LIS3DH%20Adafruit%20Schematic.pdf)
* [STM32F042K6 Datasheet](datasheets/stm32f042k6.pdf)
* [STM32F042K6 Reference Manual](datasheets/rm0091-stm32f0x1stm32f0x2stm32f0x8-advanced-armbased-32bit-mcus-stmicroelectronics.pdf)
* [STM32F042K6 Programmers Manual](datasheets/pm0215-stm32f0-series-cortexm0-programming-manual-stmicroelectronics.pdf)
* [STM32F042K6-Nucleo Schematic](datasheets/STM32F042K6-Nucleo-Schematics.pdf)

Before you begin, and we've (all) gotten a little lazy on this point, make sure your dev board is disconnected from power when you wire it up / change the wiring!

### Checkpoint 1.1 - Polling Driver Wiring

First up - which STM32F042K6-Nucleo dev board and LIS3DH dev board pins are you going to connect together?  You are constrained by the pins available on the STM32F042K6-Nucleo dev board.  Remember where to find alternate functions for pins?  Combine this information to figure out which pins on the STM32F042K6 you *must* use for SPI communications.  Fill out the table below as you sort this out.  I've put in the USART2_TX signal information as an example of the format I am looking for.  I would also take a note on which alternate function values you want - you'll need to know that when you configure the pins for SPI later.

A suggestion: you can use _any_ GPIO pin for the chip select if you want to manage chip select manually:

```
// Set chip select low
gpio_pin_reset(PORT, pin)
// Perform SPI transfer
spi_transfer(arguments)
// Set chip select high
gpio_pin_set(PORT, pin)
```

 In practice I usually manage chip select manually.  You can also use the SPI peripheral's "NSS" signal for chip select if you want to let the SPI peripheral manage chip select automatically for you.  I don't usually do this, but sometimes it works out.  It might work out in this lab... it might not.  You can defer making this decision by using the SPI peripheral's "NSS"-capable pin for chip select, and deciding later if you want to manage it manually as GPIO (digital output mode) or as SPI Peripheral Hardware NSS (alternate function mode).  

Fill out the following table with your findings.  Replace SPIx with SPI1 or SPI2 accordingly.

| STM32F042K6 Signal | STM32F042K6 GPIO Port and Pin | STM32F042-Nucleo Dev Board Connector Port and Pin |
|--------------------|-------------------------------|---------------------------------------------------|
| USART2_TX          |                 Port A Pin #2 |                                        CN4 Pin #5 |        
| SPIx_SCK           |                   PA 5        |     8                     |
| SPIx_MISO          |                   PA 6       |     7                            |
| SPIx_MOSI          |                   PA 7        |      6                     |
| SPI CS             |                   PA 4        |       9                          |

*Grading Rubric [5 points total]*:
* -1 for each incorrect entry, up to a maximum of -5

Repeat this exercise from the LIS3DH's perspective:

| STM32F042K6 Signal | LIS3DH Signal                 | LIS3DH Dev Board Connector and Pin |
|--------------------|-------------------------------|---------------------------------------------------|
| +3.3V              |                           Vdd |                                        JP3 Pin #2 |
| GND                |                           GND |                                        JP3 Pin #3 |
| SPIx_SCK           |          SCL                  |                 4                |
| SPIx_MISO          |          SDO             |                      5                      |
| SPIx_MOSI          |          SDA              |                     2 in horizontal                   |
| SPI CS             |          CS              |                      1 in horizontal                  |

*Grading Rubric [5 points total]*:
* -1 for each incorrect entry, up to a maximum of -5

Have you double-checked your findings above?  Great!  Wire it up, and power it up.


### Checkpoint 1.2 - SPI Peripheral / Accelerometer Initialization and WHO_AM_I Sanity Check

Update sysinit.c with a spi1_init() or spi2_init() (depending on your findings) function that enables clocks and configures chip IO as necessary (based on your first table, above).  No need to copy and paste code here - I'll look at your sysinit.c implementation.  

*Grading Rubric [10 points total]*:
* -2: for any missing clock enables
* -2: for each incorrectly configured SPI bus signal
* -2: inadequate commenting
* -2: "magic numbers"

Now carve out a quiet chunk of time and sit with Section 28 of the [STM32F042K6 Reference Manual](datasheets/rm0091-stm32f0x1stm32f0x2stm32f0x8-advanced-armbased-32bit-mcus-stmicroelectronics.pdf).  Do *not* try to understand everything in this section.  Do:
* Read Section 28.1
* Skip Sections 28.3 & 28.4
* Skim Section 28.5 in its entirety just to get a feel for what is in each sub-section
* Refer back to Section 28.5.X frequently throughout this lab
* Skip Section 28.6 (for now)
* Skip Sections 28.7 & 28.8 (forever, we are not doing I2S)
* Skim Sections 28.9 to get a feel for the registers
* Refer back to Section 28.9 frequently throughout this lab

Don't panic if the material in section 28.X is dense, confusing, completely opaque.  That's a common reaction to reference manuals... even for me... still... after decades with them.  When I meet a new peripheral I take a few passes at its documentation to get oriented, and then really spend time going back and forth between hardware, code, and the reference manual working out a bunch of little puzzles along the way until, eventually, it starts to come into focus.  That's just how it goes.  Don't be alarmed by it.  Do give yourself time for it. 

Review spi.h to understand what macros (#defines, etc...) are provided for you.  You should not need to add any additional structures or typedefs to spi.h, but you will need to add appropriate #define values to use when you access/modify registers.

Update spi.c's spi_init() function.  You will only be connecting a single device, the LIS3DH, to the STM32F042K6's SPI peripheral so you can set all SPI communications parameters (SPI bus speed, clock phase, clock polarity, data width) in spi_init().  You do not need to (but you are welcome to) generalize SPI read/write transfers to support multiple SPI devices on the SPI bus in this design.

*Grading Rubric [10 points total]*:
* -2: for each missing configuration setting necessary for proper functionality
* -2: inadequate commenting
* -2: "magic numbers"

Implement spi.c's spi_txfr_16() function to perform a 16-bit data transfer on the SPI bus.  You have a choice between managing the SPI chip select signal yourself (using gpio_pin_set/reset() calls), or letting the SPI hardware manage the chip select signal.  You also have a choice (if managing SPI chip select yourself) of managing the chip select in the spi_txfr_16() function call or from the accel_read() function call (next).  Document (in your code comments) your design decisions and justifications!

*Grading Rubric [6 points total]*:
* -2: failure to check if okay to transmit
* -2: failure to check if transfer complete before using results
* -2: improper data passing 
* -2: inadequate commenting
* -2: "magic numbers"
  
Next, implement a "layered" driver for the accelerometer.  This means that, from your main code (in main.c), you will only interact with accel_\*() function calls to the accelerometer driver.  Your main.c code will not directly interact with the spi_\*() function calls associated with the SPI driver.  Instead, accel_\*() function calls will call spi_\*() functions as needed.

Implement accel.c's accel_init() function.  This function is called from main() and, at present, should only call spi_init().  You will add more to this function later.

*Not graded*

Implement accel.c's accel_whoami() function.  Use the spi_txfr_16() function as part of your implementation.  This function is currently called from the systick_callback_function(), at 2 Hz, to generate a periodic read request on the SPI bus.    You will see why we want to do that in the next step.  

*Grading Rubric [4 points total]*:
* -2: wrong register accessed
* -2: result not properly masked
* -2: inadequate commenting
* -2: "magic numbers"

Ready to test and debug some code?  Let's see what is actually happening on the SPI bus.

Set up your AD3 as a logic analyzer as follows so you can see this happening on the bus:

[Analog Discovery 3 SPI Protocol Analyzer](https://dartmouth.hosted.panopto.com/Panopto/Pages/Viewer.aspx?id=0827f050-9280-4e5e-9f90-b3e301640292)

*INDIVIDUAL RESPONSE*: Show me that you have this working by replacing my image, below, with your own.  My image is not annotated, but yours should be.  Annotate the decoded hex value sent to the accelerometer and explain what the hexadecimal value represents (read transation, write transaction, what register, etc...).  Annotate the decoded hex value received from the accelerometer, explain what the hexadecimal value represents, and how you know if it is correct/incorrect:

![AD3 SPI Protocol Analyzer Results](images/proof1.png)

*Grading Rubric [10 points total]*:
* -2: logic analyzer / protocol decode results too small to be legible
* -2: value sent to accelerometer not annotated
* -2: value sent to accelerometer not explained
* -2: value received from accelerometer not annotated
* -2: value received from accelerometer not explained

## Part 2 - Configuring and Reading Acceleration Measurements

Your goal in this section is to configure the accelerometer to take high-resolution (12-bit) X,Y,Z acceleration measurements at 10 Hz.  The accelerometer has its own internal timer, so you do not need to start these acceleration measurements each time (like you did with the analog to digital converter in the last lab).  Instead, you need to configure the accelerometer.  The accelerometer is an "off-chip" peripheral (it is attached to the chip; it is not a peripheral inside of the chip).  But, just like on-chip peripherals, the accelerometer has configuration and status registers.  You interact with these registers over the SPI bus.

NOTE WELL: The LIS3DH accelerometer is not reset when you reset the STM32F042K6 microcontroller (using the debugger or the board's reset button).  The LIS3DH accelerometer's register values will remain unchanged through microcontroller resets unless you disconnect power to the board.  It is a common debugging "gotcha" to flip the "wrong" bit in e.g. an accelerometer's configuration registers, then fix your microcontroller code to flip the "right" bit but to forget to reset the accelerometer to clear the "wrong bit" by fully de-powering the board.  When in doubt, unplug the board.  Also note that you cannot necessarily get away with just disconnecting the LIS3DH's Vdd supply (by removing the jumper temporarily).  there are a number of IO pins connected the LIS3DH that may be at 3.3V, and just disconnecting Vdd may not sufficiently de-power the acceleromter enough to reset it (and may actually create more problems partially-powering it via an IO pin).  so, again, when in doubt: unplug the board to reset everything!

### Checkpoint 2.1 - Configuring and Reading Acceleration Measurements

Review the LIS3DH datasheet (again), this time paying attention to the register descriptions, while you enhance your accel_init() function to:  

* Check the WHO_AM_I register and make sure it receives a valid response.
* Configure the LIS3DH to operate in high resolution (12-bit) mode
* Configure the LIS3DH for a 10Hz measurement rate
* Enable the LIS3DH's X, Y, and Z axis acceleration measurement

Easy, right?  Unfortunately, the LIS3DH's datasheet is not particularly well written.  So you may need to experiment a bit to get this working.  Anticipate some "productive struggle" here, but don't lose a night's sleep on it.  Get hints (not answers) from other classmates if you're stuck.

*Grading Rubric [6 points total]*:
* -2: wrong registers accessed
* -2: wrong values used
* -2: inadequate commenting
* -2: "magic numbers"

Next, implement the following two utility functions defined in accel.h:

```
// Check if new accelerometer data is available.
//
// Returns true if new data is available, false if not.
bool accel_data_ready(void);
```
*Grading Rubric [6 points total]*:
* -2: wrong register accessed
* -2: wrong flag used
* -2: inadequate commenting
* -2: "magic numbers"

```
// Read out X, Y, and Z axis 16-bit signed values from
// the accelerometer.  Should be called only after 
// accel_data_read() returns 'true'. 
//
// Returns X,Y,Z acceleration measurements  from the accelerometer in units
// of "milli-Gravities," where 1 Gravity is Earth's gravitational
// acceleration (~9.8 m/s^2)
void accel_read(int16_t *x, int16_t *y, int16_t *z);
```
*Grading Rubric [10 points total]*:
* -2: wrong registers accessed
* -2: wrong computation for each of X, Y, or Z
* -2: inadequate commenting
* -2: "magic numbers"

Finally, replace your template project main()'s while(){...} loop with the following code (and remove the call to accel_whoami() from the systick_callback_handler())

```
    // Poll accelerometer for data and print to terminal when available
    while( 1 ) {
        // Add some delay between reads so we do not hammer the SPI bus
        // unnecessarily (can lock up the accelerometer as well)
        for( int i = 0; i < 50000; i++ );
        // Check if new accelerometer data is available
        if( accel_data_ready() ) {
            // If so, read it out and print it to the terminal
            int16_t x, y, z;
            accel_read(&x, &y, &z);
            printf("Accel X: %6d  Y: %6d  Z: %6d\n", x, y, z);
        }
    }
```

When everything is working correctly, your serial terminal should update ~10 times a second and, with the accelerometer flat on a desk, the output should resemble the following:

![Accel Y is negative 1 G](images/proof3.png)

*The board was on it's side perpndicular to the pins, I found the orientation by slowly changing the orientation, the x and z are bit's are non-zero, because the board isn't perfectly perpendicular to the earth, there is also some noise and uncertianty, casing them to alternate. The acceleration due to gravitiy should be 9.28 m/s^2 so the sensor is a little off*
*Grading Rubric [8 points total]*:
* -2: incorrect output for requested orientation
* -2: X output not justified 
* -2: Y output not justified 
* -2: Z output not justified 

## Part 3 - Interrupt-Driven Off-chip Peripheral Drivers

Right now you have a polling driver for your off-chip accelerometer peirpheral.  Before you can enhance your accelerometer driver to be interrupt driven, you need a way for off-chip devices to generate interrupts.  Enter the Extended Interrupts and Events (EXTI) peripheral (also sometimes referred to as the "EXTI Controller").

Watch the following video on EXTI.  There are no self-assessment / quiz questions on EXTI - you demonstrate your understanding by configuring it in the next checkpoint.

Follow along in Section 11.2 of the STM32F042K6 Reference Manual ("Extended Events and Interupts Controller").  Also be aware of sections 9.1.2 through 9.1.5 in the STM32F042K6 Reference Manual ("System Configuration" - where the GPIO pin multiplexors to map PORTx pins to the associated EXTI events live).  You will use the default PORTA Pin 0 pin for external interrupts so you do not need to modify any of the System Configuration registers in this lab.

[EXTI Overview](https://dartmouth.hosted.panopto.com/Panopto/Pages/Viewer.aspx?id=0f4d6568-3e49-47e8-8bdc-b3e4013694be)

### Checkpoint 3.1 - Interrupt-Driven Accelerometer Driver

Connect the LIS3DH Dev Board's INT1 pin to the STM32F042-Nucleo Dev Board's PORT A Pin 0 pin.

Enhance sys_spi_init() in sysinit.c to configure GPIO Port A Pin 0 as an input (or, if you do not add any code, include in your comments why not).

Enhance accel_init() in accel.c as follows:
* Configure the LIS3DH accelerometer to generate interrupts when new XYZ data is ready.  Hint: this is referred to, quite opaquely, as "ZYXDA" in the LIS3DA datasheet.
* Configure the EXTI peripheral to generate interrupts on a rising edge on PORT A Pin 0.
* Configure the NVIC to pass EXTI0 interrupts to the processing core
* Accept (a "callback") function pointer argument to a function taking three signed, 16 bit integers (X,Y,Z) as arguments but not returning any results
* "Register" a local copy of the function pointer argument passed to accel_init() in a variable visible to all functions in accel.c

*Grading Rubric [10 points possible]*
* -2: Incorrect LIS3DH interrupt configuration
* -2: Incorrect EXTI configuration
* -2: Incorrect NVIC configuration
* -2: Incorrect function pointer argument
* -2: Function pointer argument not "registered" to a variable with file scope
* -2: Insufficient commenting
* -2: Magic numbers

Create an EXTI interrupt handler in accel.c that:
* Clears the EXTI interrupt source
* Calls accel_read() to read the latest X,Y,Z acceleration values
* Passes the latest X,Y,Z acceleration values to the callback function pointer registered in accel_init()

*Grading Rubric [10 points possible]*
* -2: Does not clear the EXTI interrupt source
* -2: Does not use accel_read()
* -2: Does not check that the callback function pointer is non-null before calling
* -2: Insufficient commenting
* -2: Magic numbers

There is a tricky bit of interaction between the accelerometer's interrupt output line and using a rising-edge EXTI trigger: if the accelerometer raises the interrupt signal when the microcontroller is not watching for a rising edge (e.g. you are debugging, or you reset the microcontroller without resetting the accelerometer), the rising edge on the interrupt line will go un-detected and, once high, will not go high again when new data is ready.  As such, I add the following code to the end of my accel_init() to check if the interrupt signal from the accelerometer is "already high" and, if so, I call accel_read() to read (and discard) the data to clear the interrupt signal from the accelerometer:

```
    // If the interrupt line is already high we 
    // won't catch it because we are configured to
    // detect rising edges, not "high state", 
    // so we need to check for a high interrupt line 
    // here and, if high, read data to clear it
    if( GPIOA->IDR & gpio_pin_0 ) {
        int16_t x, y, z;
        accel_read(&x, &y, &z);
    }
```

Add the following function to main.c.  Yes yes yes I told you not to spend a lot of time in your interrupt handlers and this printf() statement is going to do just that.  Next week we'll meet DMA and feel better about this...

```
// Display latest results from the 3-axis accelerometer
void accel_callback_function(int16_t x, int16_t y, int16_t z) {
    printf("Accel X: %6d  Y: %6d  Z: %6d\n", x, y, z);
}
```

Change the call to accel_init() in main.c to pass the accel_callback_function as its argument, and modify the while(1) {...} section of main() as follows:

```
    while( 1 ) {
        ;;; // Do nothing;  all work is done in interrupt handlers
    }
```

Connect the Analog Discovery 3's digital input #4 to PORT A Pin 0 (the INT1 input from the LIS3DH), and add DIO4 to the WaveForms Logic Analyzer inputs.  Set the logic analyzer to trigger on the *rising edge* of DIO4, and adjust your timescale so your output resembles mine below (I've annotated where you click to add another signal to the logic analyzer).  Your serial terminal output should again be updating at 10 Hz, just like it did before with the polling driver, and your logic analyzer output should resemble mine:

![AD3 Triggering on INT1](images/proof2.png)

*INDIVIDUAL RESPONSE*: Explain what is going on in the logic analyzer capture above.  Include in your explanation statements about why the INT1 signal goes high (and low) when it does.  Include in your explanation what the SPI transfers that occur after the INT1 signal goes high are doing.  Explain how to decode the first two SPI transfers and what the result of that decoding represents.
on my image the DO4 represents the INT1 pin on the accelerometer, this pin is used for triggering external interupt's of the processor, it's rising edge signals that the the accelerometer has a new meassurement awaiting a read at 10Hz. Up on itnerupt the the callback function is called that calls a read, which uses the spi function to that reads the three different adresses corresponding to x, y, z acceleration. The first SPI transfer requests x acceleration and reads the first 8 bits of the x value and the second reads the other 8 bits of the x value. A8 and A9 are adresses the read representations of adresses 28 and 29 in hex.
*Grading Rubric [10 points possible]*:
-2: logic analyzer capture timescale is too big/small (cannot read the capture's results)
-2: insufficient explanation of the INT1 signal
-2: insufficient explanation about the resulting SPI transfers
-2: insufficient explanation on decoding the 1st two SPI transfers
-2: incorrect result after decoding

## Conclusion

In this week's lab you built a three axis accelerometer using a layered, interrupt driven design.  You worked with the common Serial Peripheral Interconnect (SPI) bus to connect external sensors the the microcontroller and EXTI capabilities to enable external devices to interrupt the microcontroller.  You likely spent time using the Analog Discovery 3's logic analyzer and protocol analyzer capabilities to understand, debug, and verify your systems behaviors along the way.  These are practical skills that I use regularly in design work in industry.  Well done!

When you are happy with your work, and before the assignment is due, be sure to save all files, then commit and sync (push) this project to your GitHub repo.  Unless you make other arrangements with me prior to the assignment deadline, I will grade the last sync (push) to your GitHub repo that occurs prior to the assignment deadline.  

Congratulations on finishing Week #5's lab!  Note this is a multi-week lab.  You will continue to extend this lab in Week #6 before submitting it for grading.  Be sure to save your work!